In [ ]:
%matplotlib inline

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import scipy
import pytpc
from math import ceil, floor
from pytpc.constants import *
from pytpc.tpcplot import pad_plot
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.offsetbox import AnchoredText
import matplotlib as mpl
import seaborn.apionly as sns
import glob
import re
import h5py
import yaml
import sys
from pytpc.relativity import find_proton_params
from pytpc.relativity import find_kine_vert_en
import os
from pytpc.fitting.mixins import constrain_angle, odrline

In [ ]:
with open('../fitters/config_e15503b.yml', 'r') as f:
    config = yaml.load(f)
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    patch = yaml.load(f)
config.update(patch)

In [ ]:
runtables = pd.read_csv('/Users/josh/Documents/Data/Meta/e15503b/e15503b-rundata-parseable.csv', index_col='run_num')

In [ ]:
vmedata = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/select_vme/vme_good_events_e15503b.h5')
vmedata.dropna(subset=['evt_id'], inplace=True)
vmedata.index = ['{:04g}_{:d}'.format(int(r), int(e)) for r, e in zip(vmedata.run_num, vmedata.evt_id)]

# Merge dbs

In [ ]:
rawdbs = glob.glob('/Users/josh/Documents/Data/Results/e15503b/mcmin/Apr12/raw/*.db')
print('Found {} databases'.format(len(rawdbs)))

In [ ]:
sql_query = """
    select
        m.evt_id,
        m.x0, m.y0, m.z0, m.enu0, m.azi0, m.pol0,
        m.posChi2, m.enChi2,
        m.lin_scat_ang, m.lin_beam_int, m.lin_chi2,
        m.rad_curv, m.brho, m.curv_en,
        m.curv_ctr_x, m.curv_ctr_y
    from
        mcmin_results m;
"""

In [ ]:
def read_res(db):
    with sqlite3.connect(db) as conn:
        res = pd.read_sql(sql_query, conn)
        
    run_num = re.search(r'mcmin_run_(\d+)', db).group(1)
    res['run_num'] = int(run_num)
    res['evt_id'] = res.evt_id.astype('int')
    
    res.index = ['{:04g}_{:d}'.format(r, e) for r, e in zip(res.run_num, res.evt_id)]
    return res

def read_res_all(dblist):
    return pd.concat([read_res(db) for db in dblist], axis=0)

In [ ]:
res = read_res_all(rawdbs)
print(len(res))

# Merge with VME data

In [ ]:
res = pd.merge(res, vmedata[['height', 'cfd_pos', 'pk_pos', 'group']], left_index=True, right_index=True)

In [ ]:
res.rename(columns={'height': 'ic_height', 'cfd_pos': 'ic_cfd_pos', 
                    'pk_pos': 'ic_pk_pos', 'group': 'vme_group'}, inplace=True)

# Load event ID mappings

In [ ]:
from pytpc.vmedata import VMEAlignmentTable

In [ ]:
badruns = np.array([109, 115, 140, 183, 206, 250])

In [ ]:
evtid_tables = {
    r: VMEAlignmentTable.from_hdf(f'/Users/josh/Documents/Data/Results/e15503b/align/vme_align_run_{r:04d}.h5')
    for r in res.run_num.unique().astype('int')
}

In [ ]:
invalid_ids = []
for r, table in evtid_tables.items():
    if r not in badruns:
        invalid_ids += [f'{r:04d}_{e}' for e in np.where(~table.valid)[0]]

In [ ]:
validcut = ~(res.index.isin(invalid_ids) | res.run_num.isin(badruns))

In [ ]:
assert not np.in1d(res[validcut].run_num.unique(), badruns).any()
assert not res[validcut].index.isin(invalid_ids).any()

# Scalers

In [ ]:
scalers = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/scalers/last.h5', 'last')
startstop = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/scalers/last.h5', 'startstop')

scalers = pd.merge(startstop, scalers, how='inner', left_index=True, right_index=True)
del startstop

In [ ]:
scalers['duration'] = (scalers.stop - scalers.start).astype('timedelta64[s]')

# Rutherford

In [ ]:
def rutherford(th, Z1, Z2, en):
    en = en * 1e6 * e_chg
    return (Z1*Z2*e_chg**2 / (16*pi*eps_0*en))**2 / np.sin(np.pi/2 - th)**4 * 1e28

# Post-processing

In [ ]:
def z_from_vert_en(vert_en, beam_enu0, beam_mass, beam_charge, gas):
    ef = vert_en * beam_mass
    ei = beam_enu0 * beam_mass
    ri = gas.range(ei, beam_mass, beam_charge)  # m
    rf = gas.range(ef, beam_mass, beam_charge)  # m
    
    return 1 + (rf - ri)  # m

In [ ]:
gas = pytpc.gases.InterpolatedGas(config['gas_name'], config['gas_pressure'])

In [ ]:
from pytpc.utilities import find_vertex_energy

def postprocess(test_res, gas):
    test_res['scat_ang'] = pi - test_res.pol0
    test_res['cm_angle'] = pi - 2*test_res.scat_ang  # Approximation
    test_res['totChi2'] = test_res['posChi2'] + test_res['enChi2']

    test_res['kine_vert_en'] = find_kine_vert_en(46*p_mc2, p_mc2, test_res['scat_ang'], test_res.enu0) / 46
    test_res['vert_en'] = find_vertex_energy(test_res.z0, config['beam_enu0'], 46, 18, gas) / 46
    test_res['endiff'] = test_res.kine_vert_en - test_res.vert_en

In [ ]:
postprocess(res, gas)

# Cuts

In [ ]:
prodcut = res.run_num.isin(runtables[runtables.type == 'Production'].index)

In [ ]:
posChi2Max = 14
enChi2Max = 10

In [ ]:
with sns.plotting_context('paper'):
# with sns.plotting_context('notebook'):
    fig, ax = plt.subplots(1, 2, squeeze=True, sharey=True, 
                           figsize=(6, 2.5),
#                            figsize=(12, 4),
                          )

    ax[0].hist(res.posChi2.dropna().values, bins=400, range=(0, 99), histtype='stepfilled');
    ax[0].set_xlabel(r'$\chi^2_\mathrm{pos}$')
    ax[0].set_ylabel('Count')

    ax[1].hist(res.enChi2.dropna().values, bins=400, range=(0, 50), histtype='stepfilled');
    ax[1].set_xlabel(r'$\chi^2_\mathrm{en}$')

    for a, bd in zip(ax.ravel(), (posChi2Max, enChi2Max)):
        a.set_autoscalex_on(False)
        a.set_autoscaley_on(False)
        a.axvspan(0, bd, facecolor='#dcdee2', zorder=0)
        a.vlines((0, bd), *a.get_ylim(), linewidth=1)
        a.yaxis.get_major_locator().set_params(nbins=6)
        a.xaxis.get_major_locator().set_params(nbins=8)

    fig.tight_layout()
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/expchi2.pdf', bbox_inches='tight')


In [ ]:
with sns.plotting_context('notebook'):
#     plt.figure(figsize=(6, 4))

    bins, edges = np.histogram(res.enChi2.values, bins=100, range=(0, 70));

    plt.step(edges[:-1], bins, color='k', label='Energy $\chi^2$')
    plt.fill_between(edges[:-1], bins, step='pre', where=edges[:-1] < enChi2Max, facecolor='C0', edgecolor='none',
                     label='Proton tracks')
    plt.fill_between(edges[:-1], bins, step='pre', where=(edges[:-1] > 13) & (edges[:-1] < 36), 
                     edgecolor='none', facecolor='C3', label='Carbon tracks')

    plt.xlim(0, 70)
    plt.ylim(0)
    # plt.vlines(enChi2Max, *plt.ylim())
    # plt.axvspan(0, enChi2Max, zorder=-1, facecolor=)
    # plt.axvspan(12, 40, facecolor='none', hatch='//')
    sns.despine()
    plt.xlabel(r'$\chi^2_\mathrm{en}$')
    plt.ylabel('Count')

    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

    plt.legend()
    plt.tight_layout()

#     plt.savefig('/Users/josh/Desktop/Figures for committee/enchi.pdf', bbox_inches='tight')

In [ ]:
physicalcut = (res.scat_ang < pi/2)
goodfitcut = (res.posChi2 < posChi2Max) & (res.enChi2 < enChi2Max)

In [ ]:
print('goodfitcut: {} / {} = {}'.format(len(res[goodfitcut]), len(res), len(res[goodfitcut])/len(res)))

In [ ]:
len(res)

In [ ]:
len(res[goodfitcut & physicalcut & prodcut])

In [ ]:
recalib_scale = 1.09
res['vert_en_recalib'] = 4.17 - (4.17 - res.vert_en)*recalib_scale

In [ ]:
gooddata = res[goodfitcut & physicalcut & prodcut & validcut]

In [ ]:
len(gooddata)

# Plots 

In [ ]:
with sns.plotting_context('paper'):
    max_theta = 65
    d = gooddata[gooddata.scat_ang <= max_theta * degrees].sort_values('z0')
    
    plt.figure(figsize=(5, 3))

    plt.hist2d(d.z0 * 1000, d.enu0 / np.cos(d.scat_ang)**2, bins=(np.linspace(0, 1000, 100), np.linspace(0, 20, 100)),
               alpha=0.7, cmap='Greys')

    recalib_lb = 4.17 - (4.17 - d.vert_en)*(recalib_scale - 0.025)
    recalib_ub = 4.17 - (4.17 - d.vert_en)*(recalib_scale + 0.025)

    plt.plot(d.z0 * 1000, d.vert_en * 4, linestyle='--', color='tab:red', label='Original')
    plt.plot(d.z0 * 1000, d.vert_en_recalib * 4, linestyle='-', color='tab:cyan', label='Recalibrated')
    plt.fill_between(d.z0 * 1000, recalib_lb * 4, recalib_ub * 4, color='tab:cyan', alpha=0.3)

    plt.colorbar().set_label('Count')
    plt.xlabel(r'Vertex position $z_0$ [mm]')
    plt.ylabel(r'$E_p / \cos^2(\theta_\mathrm{lab})$ [MeV]')
    plt.legend(loc='upper left', frameon=False)
    plt.gca().add_artist(AnchoredText(fr'$\theta_\mathrm{{lab}} \leq {max_theta}°$', loc=4, frameon=False))

#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/recalib.pdf', bbox_inches='tight', dpi=400)

In [ ]:
with sns.plotting_context('paper'):
    fig, ax_main = plt.subplots(figsize=(3.5, 2.6))
    
    binsize = 5
    
    ax_main.hist(gooddata.z0 * 1000, bins=np.arange(0, 1700, binsize), histtype='stepfilled');

    ax_main.set_xlabel('Vertex position in z [mm]')
    ax_main.set_ylabel('Count')

    ax_sub = fig.add_axes([0.6, 0.35, 0.3, 0.4])
    
    xb, xe = np.histogram(gooddata.z0 * 1000, bins=np.arange(900, 1050, binsize))
    
    from scipy.stats import norm
    def fitfunc(x, mean, sigma, scale, offset):
        return scale * (1 - norm.cdf(x, loc=mean, scale=sigma)) + offset

    fparam, ferr = scipy.optimize.curve_fit(fitfunc, xe[:-1], xb, p0=(1000, 5, 650, 0))
    print(fparam)
    print('FWHM = {:0.4f} mm'.format(fparam[1] * 2.355))
    
    xs = np.linspace(xe.min(), xe.max(), 1000)
    ax_sub.errorbar(xe[:-1], xb, yerr=np.sqrt(xb), fmt='.', elinewidth=1, capthick=1, capsize=1)
    ax_sub.plot(xs, fitfunc(xs, *fparam), 'k')
    ax_sub.set_xlim(xe.min(), xe.max())
    ax_sub.set_xticks(range(940, 1050, 40))
    ax_sub.set_yticks([])
    
    sns.despine()

    # plt.tight_layout()
#     plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/z_dist.pdf', bbox_inches='tight')
#     plt.savefig('/Users/josh/Desktop/z_dist.pdf', bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'):
    fig, ax_main = plt.subplots(figsize=(5, 5*2/3))
    
    binsize = 10e-3

    ax_main.set_xlabel('Vertex energy from vertex position (lab) [MeV/u]')
    ax_main.set_ylabel('Counts')
    
    xb, xe = np.histogram(gooddata.vert_en_recalib, bins=np.arange(3.8, 4.5, binsize))
    
    from scipy.stats import norm
    def fitfunc(x, mean, sigma, scale, offset):
        return scale * (1 - norm.cdf(x, loc=mean, scale=sigma)) + offset

    fparam, ferr = scipy.optimize.curve_fit(fitfunc, xe[:-1], xb, p0=(4.2, 40e-3, 400, 0))
    print(fparam)
    print('FWHM = {:0.4f} keV'.format(fparam[1] * 2.355 * 1000))
    
    xs = np.linspace(xe.min(), xe.max(), 1000)
    ax_main.errorbar(xe[:-1], xb, yerr=np.sqrt(xb), fmt='.', elinewidth=1, label='Data', zorder=100)
    ax_main.plot(xs, fitfunc(xs, *fparam), label='Fit')
    ax_main.set_xlim(xe.min(), xe.max())
#     ax_sub.set_xticks(range(940, 1050, 40))
    
    plt.legend(frameon=False)
    
    sns.despine()

    plt.tight_layout()
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/window_en_fit.pdf', bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'):
    plt.figure(figsize=(5, 5*2/3))   
    plt.hist(gooddata.enu0, histtype='stepfilled', bins=np.arange(0, 12.1, 0.05), log=True);
    plt.xlabel('Proton initial energy (lab) [MeV]')
    plt.ylabel('Count')
    plt.ylim(0.5)
    plt.minorticks_on()

    sns.despine()

#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/proton_energy.pdf', bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'):
    plt.figure(figsize=(5, 5*2/3))
    plt.hist(gooddata.cm_angle / degrees, bins=np.arange(0, 181, 1), histtype='stepfilled');
    plt.xlabel('Scattering angle (CM) [deg]')
    plt.ylabel('Count')
    
    plt.gca().xaxis.get_major_locator().set_params(nbins=8, steps=[1, 2, 3, 6, 10])
    plt.minorticks_on()

    sns.despine()
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/cm_angle.pdf', bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'):
    plt.figure(figsize=(4, 2.5))
    plt.hist(gooddata.azi0 / degrees, bins=np.arange(-180, 181, 1), histtype='stepfilled');
    plt.xlabel('Scattering angle (CM) [deg]')
    plt.ylabel('Count')
    
    plt.gca().xaxis.get_major_locator().set_params(nbins=8, steps=[1, 2, 3, 6, 10])
    plt.minorticks_on()

    sns.despine()
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/cm_angle.pdf', bbox_inches='tight')

In [ ]:
d = gooddata
plt.hist(d.run_num, bins=np.arange(d.run_num.min(), d.run_num.max()))
plt.xlabel('Run number')
plt.ylabel('Number of good events')

In [ ]:
d = gooddata
plt.hist2d(d.run_num, d.endiff, bins=(np.arange(d.run_num.min(), d.run_num.max(), 1), 
                                      np.arange(-2, 2, 0.1)), cmap='viridis');
# plt.axvline(217, color='white', linewidth=1)
plt.xlabel('Run number')
plt.ylabel('Energy difference [MeV/u]')
# plt.ylim(0.1, 1e4)
# plt.xlim(150, 175)

In [ ]:
argon_ct = pd.read_table('/Users/josh/Documents/Data/Results/e15503b/select_vme/argon_count.tsv', 
                         index_col=0).sort_index()
argon_ct['pct'] = argon_ct.argon_count / argon_ct.total_count

In [ ]:
inrange = gooddata[(gooddata.endiff < -0.5) & (gooddata.endiff > -1.5)]

m1cts = inrange.groupby('run_num').agg({'evt_id': 'count'}).rename(columns={'evt_id': 'inrange'})
allcts = gooddata.groupby('run_num').agg({'evt_id': 'count'}).rename(columns={'evt_id': 'total'})
ctsdf = pd.concat((m1cts, allcts), axis=1, join='inner')

del inrange, allcts, m1cts

ctsdf['ratio'] = ctsdf.inrange / ctsdf.total

In [ ]:
ctsdf = ctsdf.merge(scalers, left_index=True, right_index=True, how='inner')

In [ ]:
ctsdf['argon_pct'] = argon_ct.pct

In [ ]:
plt.bar(ctsdf.index, ctsdf.ratio, width=1)

In [ ]:
plt.scatter(ctsdf.ratio, ctsdf.argon_pct, alpha=0.6, s=20)

In [ ]:
plt.bar(ctsdf.index, ctsdf.inrange / ctsdf.cobo_or, width=1)
# plt.xlim(125, 150)

In [ ]:
armruns = ctsdf[ctsdf.ratio > 0.2].index.values

In [ ]:
armruns

In [ ]:
plt.hist(gooddata.ic_cfd_pos, bins=np.arange(0, 512, 1), log=True);

m = res.ic_cfd_pos.median()
s = 6

plt.autoscale(False)
plt.vlines((m - s, m + s), *plt.ylim())
print(m-s, m+s)

In [ ]:
iccut = (res.ic_cfd_pos > 145) & (res.ic_cfd_pos < 157)

In [ ]:
res[goodfitcut & prodcut & physicalcut & iccut].shape

In [ ]:
res[goodfitcut & prodcut & physicalcut & ~iccut].shape

In [ ]:
d = res[goodfitcut & prodcut & physicalcut]
plt.hist2d(d.ic_cfd_pos, d.endiff, bins=(np.arange(512), np.arange(-2, 2, 0.1)), norm=LogNorm());
plt.colorbar()

In [ ]:
with sns.plotting_context('paper'):
    d = gooddata
    
    jg = sns.JointGrid(d.x0.values * 1000, d.y0.values * 1000, size=3.75)
    
    jg.plot_joint(plt.hist2d, bins=np.arange(-20, 20.5, 0.5))
    jg.plot_marginals(sns.distplot, hist=True, kde=False, bins=np.arange(-20, 20.5, 0.5), 
                      color=plt.cm.viridis(0.25), hist_kws={'histtype': 'stepfilled'})
    
    jg.set_axis_labels('$x_0$ [mm]', '$y_0$ [mm]')
    
    mbox = jg.ax_marg_y.get_position()
    cax = jg.fig.add_axes([mbox.x1 + 0.05, mbox.y0, mbox.height / 20, mbox.height])
    plt.colorbar(cax=cax).set_label('Counts')
    
    jg.ax_joint.xaxis.get_major_locator().set_params(nbins=8)
    jg.ax_joint.yaxis.get_major_locator().set_params(nbins=8)
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/vertex_xy.pdf', bbox_inches='tight', dpi=400)

In [ ]:
with sns.plotting_context('paper'):
    d = gooddata
    
    plt.figure(figsize=(4.75, 3))
    plt.hist2d(d.z0 * 1000, np.hypot(d.x0, d.y0) * 1000, bins=(np.arange(0, 1010, 10), np.arange(0, 30.05, 0.5)), 
               norm=LogNorm());
    plt.colorbar().set_label('Counts')

    plt.ylim(0, 30.1)

    plt.xlabel('$z_0$ [mm]')
    plt.ylabel('$\sqrt{x_0^2 + y_0^2}$ [mm]')
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/vertex_r_vs_z.pdf', bbox_inches='tight', dpi=400)

In [ ]:
gsrescut = ((res.cm_angle > 70 * degrees) & (res.cm_angle < 120 * degrees) 
            & (res.vert_en < 2.2) & (res.vert_en > 1.5))

In [ ]:
with sns.plotting_context('notebook'):
    d = gooddata
    plt.hist2d(d.cm_angle/degrees, d.vert_en_recalib, range=((10, 140), (0, 6)), bins=100, norm=LogNorm());
    plt.xlabel('CM scattering angle [deg]')
    plt.ylabel('Reconstructed 46Ar vertex energy [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
d = gooddata
plt.hist(d.endiff, bins=np.arange(-5, 5, 0.05), histtype='stepfilled');
# vertdiffmed = -1 #d.vertdiff.median()
# sigma = 0.200

# vertencut = np.abs(res.vertdiff - vertdiffmed) < sigma

# plt.vlines((vertdiffmed, vertdiffmed - sigma, vertdiffmed + sigma), *plt.ylim())
# plt.ylim(0.1, 1e4)
plt.xlabel('(Kine. vertex en.) – (pos. vertex en.) [MeV/u]')
plt.ylabel('Count')

# plt.savefig('/Users/josh/Desktop/Plots for Daniel/exp_endiff.pdf')

In [ ]:
from scipy.stats import norm, laplace, gennorm
from scipy.optimize import curve_fit

def gaus(x, s, a):
    return a * poisson.pdf(x, loc=0, scale=s)

def lorentz(x, x0, gamma, a):
    return a / ((x - x0)**2 + 0.25 * gamma**2)

def lapl(x, ctr, width, scale):
    return scale * laplace.pdf(x, loc=ctr, scale=width)

def gen(x, ctr, width, scale, beta):
    return scale * gennorm.pdf(x, beta=beta, loc=ctr, scale=width)

In [ ]:
fitdata = gooddata

bins = np.arange(-5, 5.01, 0.05)
fitbins = bins #bins[bins >= -0.5]

fitcts, fitedges = np.histogram(fitdata.endiff, bins=fitbins)

fparam, fres = curve_fit(lorentz, fitedges[:-1], fitcts)#, p0=(0, 1, fitcts.max(), 2))
print(fparam)

plt.step(fitedges[:-1], fitcts, where='post')

xs = np.linspace(-5, 5, 1000)
plt.plot(xs, lorentz(xs, *fparam))

plt.xlabel('(Kine. vertex en.) – (pos. vertex en.) [MeV/u]')
plt.ylabel('Count')

In [ ]:
plt.hist(fitdata.endiff, bins=bins);

xs = np.linspace(-5, 5, 500)
plt.plot(xs, lorentz(xs, *fparam))

In [ ]:

cts, edges = np.histogram(fitdata.endiff, bins=bins)

plt.step(edges[:-1], cts - lorentz(edges[:-1], *fparam))
plt.vlines((-1.15), *plt.ylim())

In [ ]:
d = res[goodfitcut & physicalcut & vertencut]
plt.hist(d.vertdiff, bins=100, log=False, range=(-5, 5));

plt.vlines((vertdiffmed, vertdiffmed - sigma, vertdiffmed + sigma), *plt.ylim())
# plt.ylim(0.1, 1e4)
plt.xlabel('(Kine. vertex en.) – (pos. vertex en.) [MeV/u]')
plt.ylabel('Count')

In [ ]:
print("Vert en cut keeps {} / {}".format(len(res[goodfitcut & physicalcut & vertencut]), 
                                         len(res[goodfitcut & physicalcut])))

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('white'):
    e = 0.5
    d = gooddata
    fig, ax = plt.subplots(1, 2, squeeze=True, figsize=(18, 6))
    *junk, c0 = ax[0].hist2d(d.cm_angle/degrees, d.vert_en, range=((10, 140), (0, 6)), bins=100, cmap='viridis')#, norm=LogNorm());
    *junk, c1 = ax[1].hist2d(d.cm_angle/degrees, d.kine_vert_en, range=((10, 140), (0, 6)), bins=100, cmap='viridis')#, norm=LogNorm());
    plt.colorbar(c0, ax=ax[0]).set_label('Count')
    plt.colorbar(c1, ax=ax[1]).set_label('Count')
    ax[0].set_xlabel('CM scattering angle [deg]')
    ax[1].set_xlabel('CM scattering angle [deg]')
    ax[0].set_ylabel('46Ar vertex energy [MeV/u]')
    
    ax[0].set_title('From vertex position')
    ax[1].set_title('From kinematics')
    

#     plt.savefig('/Users/josh/Desktop/Figures for committee/exp_pos_2d.pdf', transparent=True, bbox_inches='tight')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & physicalcut & prodcut & (res.vme_group == 0)]
    plt.hist2d(d.ic_height, d.endiff, bins=100, range=((4500, 6500), (-10, 10)), cmap='viridis', norm=LogNorm());
    plt.xlabel('IC signal amplitude')
    plt.ylabel('46Ar Vertex energy difference [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
plt.hist(gooddata.kine_vert_en, bins=np.arange(0, 8.1, 0.05));
plt.xlabel('46Ar lab vertex energy from kinematics [MeV/u]')
plt.ylabel('Count')

In [ ]:
plt.hist(gooddata.vert_en, bins=np.arange(0, 4.8, 0.05));
plt.xlabel('46Ar energy at reaction vertex [MeV/u]')
plt.ylabel('Count')
sns.despine()
# plt.savefig('/Users/josh/Desktop/Figures/energy_spectrum.pdf', transparent=True, bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'):
    d = gooddata

    fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, squeeze=True, figsize=(5, 5*2/3*1.8))

    bins = np.arange(0, 7, 0.05)

    ax[0].hist(d.vert_en_recalib, bins=bins, histtype='stepfilled')
    ax[1].hist(d.kine_vert_en, bins=bins, histtype='stepfilled')

    ax[1].set_xlabel('${}^{46}\mathrm{Ar}$ vertex energy (lab) [MeV/u]')
    ax[0].set_ylabel('Counts')
    ax[1].set_ylabel('Counts')

    ax[0].xaxis.get_major_locator().set_params(nbins=10)

    ax[0].text(0.98, 0.95, 'From position', transform=ax[0].transAxes, ha='right', va='top')
    ax[1].text(0.98, 0.95, 'From kinematics', transform=ax[1].transAxes, ha='right', va='top')
    
    plt.minorticks_on()

    plt.tight_layout()

#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/vertens.pdf', bbox_inches='tight')

In [ ]:
ens = np.arange(1.0, 3.25, 0.25)
angs = np.linspace(50, 80, 1000) * degrees

with sns.cubehelix_palette(len(ens), start=.5, rot=-0.75, light=0.7) as pal, sns.plotting_context('paper'):
    plt.figure(figsize=(6, 4))
    
    for en in ens:
        val = find_kine_vert_en(46*p_mc2, p_mc2, angs, en) / 46
        der = np.diff(val) / (np.diff(angs) / degrees)
        plt.plot(angs[1:] / degrees, der)
        
    sns.despine()
        
    cmap = mpl.colors.ListedColormap(pal) 
    sm = mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(ens.min(), ens.max()), cmap=cmap)
    sm._A = []
    plt.colorbar(sm, ticks=ens).set_label('Proton energy [MeV]')
    plt.xlabel('Scattering angle (lab) [deg]')
    plt.ylabel(r'$dE_\mathrm{Ar}/d\theta$ [(MeV/u)/deg]')
    
    plt.minorticks_on()
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/angdep.pdf', bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper'):
    d = gooddata
    plt.figure(figsize=(7, 5))
    plt.hist2d(d.vert_en_recalib, d.kine_vert_en, bins=np.arange(0, 8, 0.05), norm=LogNorm());
    plt.gca().set_aspect(1)
    plt.xlabel('Vertex energy from vertex position (lab) [MeV/u]')
    plt.ylabel('Vertex energy from kinematics (lab) [MeV/u]')
    plt.colorbar().set_label('Counts')
    xs = np.linspace(0, 8, 10)
    plt.plot(xs, xs, 'k--', linewidth=1)
    
    plt.xticks(np.arange(9))
    plt.yticks(np.arange(9))
    
    plt.xlim(0, 8)
    plt.ylim(0, 8)
    
    plt.minorticks_on()
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/verten_2d.pdf', bbox_inches='tight', dpi=400)

In [ ]:
with sns.plotting_context('paper'):
    d = gooddata
    print(len(d))
    
    ni_mass = 60
    ni_beam_en = (4.6 * ni_mass - 20) / ni_mass

    v = find_vertex_energy(d.z0, ni_beam_en, ni_mass, 28, gas) / ni_mass
    k = find_kine_vert_en(ni_mass*p_mc2, p_mc2, d['scat_ang'], d.enu0) / ni_mass
    
#     plt.figure(figsize=(7, 5))
    plt.hist2d(v, k, bins=np.arange(0, 8, 0.05), norm=LogNorm());
    plt.gca().set_aspect(1)
    plt.xlabel('${}^{60}\mathrm{Ni}$ Vertex energy from vertex position [MeV/u]')
    plt.ylabel('${}^{60}\mathrm{Ni}$ Vertex energy from kinematics [MeV/u]')
    plt.colorbar().set_label('Counts')
    xs = np.linspace(0, 8, 10)
    plt.plot(xs, xs, 'k--', linewidth=1)
    
    plt.xticks(np.arange(9))
    plt.yticks(np.arange(9))
    
    plt.xlim(0, 8)
    plt.ylim(0, 8)
    
#     plt.savefig('/Users/josh/Desktop/energy_comparison_60ni.pdf', bbox_inches='tight', dpi=300)

In [ ]:
d = gooddata
plt.hist2d(d.cm_angle / degrees, d.endiff, range=((0, 180), (-4, 4)), bins=100, cmap='viridis', norm=LogNorm());
plt.xlabel('CM scattering angle [deg]')
plt.ylabel('Kine vert en - pos vert en [MeV/u]')
plt.colorbar().set_label('Count')

In [ ]:
d[np.abs(d.z0 - 1.0) < 1e-3].vert_en

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(d.z0, d.endiff, range=((0, 1.1), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Vertex Z position [m]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    plt.hist2d(d.enu0, d.endiff, range=((0, 5), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Proton energy [MeV]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar().set_label('Count')

In [ ]:
with sns.axes_style('white'):
    d = res[goodfitcut & prodcut & physicalcut]
    
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 6))
    plt.sca(ax[0])
    plt.hist2d(d.vert_en, d.endiff, range=((0, 8), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Pos vert en [MeV/u]')
    plt.ylabel('Kine vert en - pos vert en [MeV/u]')
    plt.colorbar(orientation='horizontal').set_label('Count')
    
    plt.sca(ax[1])
    plt.hist2d(d.kine_vert_en, d.endiff, range=((0, 8), (-4, 4)), bins=100, cmap='viridis');
    plt.xlabel('Kine vert en [MeV/u]')
    plt.colorbar(orientation='horizontal').set_label('Count')
    
    plt.tight_layout()

In [ ]:

d = res[goodfitcut & prodcut & physicalcut]

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 6))
plt.sca(ax[0])
plt.hist2d(d.vert_en, d.z0, range=((0, 8), (0, 1.1)), bins=100, cmap='viridis');
plt.xlabel('Pos vert en [MeV/u]')
plt.ylabel('Vertex Z position [m]')
plt.colorbar(orientation='horizontal').set_label('Count')

plt.sca(ax[1])
plt.hist2d(d.kine_vert_en, d.z0, range=((0, 8), (0, 1.1)), bins=100, cmap='viridis');
plt.xlabel('Kine vert en [MeV/u]')
plt.colorbar(orientation='horizontal').set_label('Count')

plt.tight_layout()

# plt.savefig('/Users/josh/Desktop/z_vs_en.pdf', dpi=400)

In [ ]:
with sns.axes_style('white'):
    dtheta = 1.0
    dE = 50e-3
    
    min_vert_en = 3.0
    delta_vert_en = 0.05
    
    histbins = (np.arange(0, 100 + dtheta, dtheta),
                np.arange(0, 6 + dE, dE))
    d = res[goodfitcut
            & (res.vert_en > min_vert_en) & (res.vert_en < min_vert_en + delta_vert_en)
           ]
    plt.hist2d(d.scat_ang / degrees, d.enu0, cmap='viridis', bins=histbins, norm=LogNorm());
    plt.colorbar().set_label('Count')
    plt.xlabel('CM Angle [deg]')
    plt.ylabel('Proton energy (lab) [MeV]')
    
    ths = np.linspace(0, pi/2, 180)
    p_ens = pytpc.relativity.find_proton_params(ths, 46*p_mc2, p_mc2, p_mc2, 46*p_mc2, 46*min_vert_en)[1] - p_mc2
    cm_angs = pi - 2*ths
    plt.plot(ths/degrees, p_ens, 'k-', linewidth=0.5, alpha=1)

In [ ]:
ens = np.linspace(0, 4.6, 1000)
plt.plot(ens, gas.energy_loss(ens * 46, 46, 18))

In [ ]:
gas.density

In [ ]:
gas.range(4.6*46-20, 46, 18) - gas.range(3.67*46 - 20, 46, 18)

---
# Binning

In [ ]:
import xarray as xr

In [ ]:
dtheta = 5.0
dE = 20 * 1e-3
thbins = np.arange(0, 130 + dtheta, dtheta, dtype='int')
enbins = np.arange(0, 5 + dE, dE)

def bin_data(data, thbins, enbins, enname='vert_en', angname='cm_angle'):
    run_nums = data.run_num.unique()
    binned = xr.DataArray(
        data=np.full((len(run_nums), len(enbins) - 1, len(thbins) - 1), np.nan),
        dims=('run_num', 'vert_en', 'cm_angle'),
        coords={'run_num': run_nums, 'vert_en': enbins[:-1], 'cm_angle': thbins[:-1]},
        name='binned_counts',
    )
    
    data = data[
        (data[angname] > thbins.min())
        & (data[angname] < thbins.max())
        & (data[enname] > enbins.min())
        & (data[enname] < enbins.max())
    ]
    
    for run_num in run_nums:
        rundata = data[data.run_num == run_num]
        b, x, y = np.histogram2d(rundata[enname], rundata[angname] / degrees, bins=(enbins, thbins))
        binned.loc[dict(run_num=run_num)] = b
    
    return binned

In [ ]:
binned = bin_data(gooddata, thbins, enbins, enname='vert_en_recalib')

In [ ]:
mcres_h5 = os.path.join(os.path.dirname(os.path.dirname(rawdbs[0])), 'mcresults.h5')
print(mcres_h5)

In [ ]:
# with h5py.File(mcres_h5, 'w') as hf:
#     hf['binned_res_array'] = binned.values
#     hf['enbins'] = binned.vert_en
#     hf['thbins'] = binned.cm_angle
#     hf['run_nums'] = binned.run_num

---

In [ ]:
df = binned.sum('run_num').to_dataframe().reset_index()
df['err'] = np.sqrt(df.binned_counts)

In [ ]:
df['z0'] = z_from_vert_en(df.vert_en, config['beam_enu0'], config['beam_mass'], config['beam_charge'], gas)

In [ ]:
# df.to_csv('/Users/josh/Desktop/data_with_z0.csv', index=False)

In [ ]:
with sns.plotting_context('paper'):
    fg = sns.FacetGrid(df, col='cm_angle', col_order=np.arange(40, 100, 5), col_wrap=3, sharey=False, size=2.1/1.2,
                       aspect=1.2)
    fg.map(plt.errorbar, 'vert_en', 'binned_counts', 'err', fmt='.', markersize=3, elinewidth=0.5)
    fg.set_axis_labels('Vertex energy [MeV/u]', 'Counts')
    fg.set_titles(r'$\theta_\mathrm{{CM}} = {col_name}^\circ$')

    plt.tight_layout()
    for ax in fg.axes.ravel():
    #     ax.set_xlim(2, 4)
        ax.set_xticks(np.arange(6))
        ax.yaxis.get_major_locator().set_params(nbins=6)
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/counts_slices.pdf')

In [ ]:
with sns.plotting_context('paper'):
    d = binned.sum('run_num').values
    X, Y = np.meshgrid(binned.cm_angle, binned.vert_en)
    
    plt.figure(figsize=(4.5, 4.5*2/3))
    plt.pcolormesh(X, Y, d, norm=LogNorm(), rasterized=True)
    plt.xlabel('Scattering angle (CM) [deg]')
    plt.ylabel(r'${}^{46}\mathrm{Ar}$ vertex energy (lab) [MeV/u]')
    plt.minorticks_on()

    plt.colorbar().set_label('Counts')
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/counts_2d.pdf', dpi=400, bbox_inches='tight')

# Smoothed BGs

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVR
from scipy.stats import uniform, lognorm, halfnorm, norm, f as fstat
from scipy.interpolate import InterpolatedUnivariateSpline
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
from effsim.paramgen import parse_dsigmaiv_output

def get_rmat(path):
    ens, angs, xsecs = parse_dsigmaiv_output(path)
    return xr.DataArray(
        data=xsecs, 
        coords={'vert_en': ens * 47/46, 'cm_angle': np.round(angs / degrees).astype('int')}, 
        dims=('vert_en', 'cm_angle')
    )
    
# rmat = get_rmat('/Users/josh/Documents/Code/ar40-aug15/rmatrix/DSIGMAIV-ELAS.DAT')
rmat = get_rmat('/Volumes/analysis/attpc/dsigma4/46Ar/DSIGMAIV-ELAS.DAT')
# rmat_phase = get_rmat('/Volumes/analysis/attpc/dsigma4/resints/5kev/DSIGMAIV-ELAS.DAT')
# rmat_phase = get_rmat('/Volumes/analysis/attpc/dsigma4/46Ar-mixphase/DSIGMAIV-ELAS.DAT')
rmat_phase = get_rmat('/Volumes/analysis/attpc/dsigma4/46Ar-4res/DSIGMAIV-ELAS.DAT')
rmat_bg = get_rmat('/Volumes/analysis/attpc/dsigma4/46Ar-nores/DSIGMAIV-ELAS.DAT')

In [ ]:
# d = binned.sum('run_num').loc[dict(cm_angle=angle)]
# d = d[np.where((d.vert_en <= 4) & (d.vert_en >= 1))]
# d = binned.sum('run_num')
# d = d.where((d.cm_angle >= 0) & (d.vert_en <= 4.09) & (d.vert_en >= 2), drop=True).sum('cm_angle')

In [ ]:
# all_binned = binned.sum('run_num')
# all_binned = all_binned.where((all_binned.vert_en <= 4.08) & (all_binned.vert_en >= 2), drop=True)

# poly = PolynomialFeatures(degree=2)
# X_ = all_binned.vert_en.values.reshape(-1, 1)

# data_model = LinearRegression(fit_intercept=False)

# pipeline = Pipeline([('poly', poly), ('ridge', data_model)])

# Y_ = all_binned.sum('cm_angle')

# pipeline.fit(X_, Y_)

# data_baseline = pipeline.predict(X_)
# d = (Y_ - data_baseline) / data_baseline
# derr = np.sqrt(Y_) / data_baseline

# print(data_model.coef_, data_model.intercept_)

In [ ]:
def pred_bounds(pred_res, X, alpha):
    pred_std = np.sqrt(np.diag(X_ @ pred_res.cov_params() @ X_.T))
    
    pred_Y = pred_res.predict(X)
    
    tval = scipy.stats.t.isf(alpha/2, pred_res.df_resid)
    
    lb = pred_Y - tval * pred_std
    ub = pred_Y + tval * pred_std
    
    return pred_std, lb, ub

In [ ]:
all_binned = binned.sum('run_num')
all_binned = all_binned.where((all_binned.vert_en <= 4.0) & (all_binned.vert_en >= 2), drop=True)

x = all_binned.vert_en.values
X_ = np.column_stack((x, x**2))
X_ = sm.add_constant(X_)

Y_ = all_binned.sum('cm_angle')

data_model = sm.OLS(Y_.values, X_)
data_fit_result = data_model.fit()

# data_prstd, data_int_lb, data_int_ub = wls_prediction_std(data_fit_result, alpha=1-0.68)
data_prstd, data_int_lb, data_int_ub = pred_bounds(data_fit_result, X_, alpha=1-0.68)

data_baseline = data_fit_result.predict(X_)
d = (Y_ - data_baseline) / data_baseline

sigma_data_fit = (data_int_lb - data_int_ub) / 2
derr = (d+1) * np.sqrt(1 / Y_ + (sigma_data_fit / data_baseline)**2)

data_fit_result.summary(xname=('const', 'x', 'x^2'))

In [ ]:
x = all_binned.sum('cm_angle')

with sns.plotting_context('talk'):
#     plt.figure(figsize=(4, 4*2/3))
    plt.errorbar(x.vert_en, x.values, np.sqrt(x.values), fmt='.', elinewidth=0.5, label='Experiment', zorder=200)
    plt.plot(x.vert_en, data_baseline, label='Quad. fit', zorder=100)
    plt.fill_between(x.vert_en, data_int_lb, data_int_ub, color='C1', alpha=0.2, zorder=99)
#     plt.plot(x.vert_en, data_int_lb, 'C1--', zorder=99)
#     plt.plot(x.vert_en, data_int_ub, 'C1--', zorder=99)

    plt.legend(loc=4, frameon=False)

    plt.xlabel('${}^{46}\mathrm{Ar}$ vertex energy (lab) [MeV/u]')
    plt.ylabel('Counts')
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/data_bgfit.pdf', bbox_inches='tight')

In [ ]:
plt.errorbar(d.vert_en, d.values, derr, fmt='.', elinewidth=0.5)
plt.axhline(0, color='k', zorder=0, linewidth=1)
# plt.ylim(-1, 3)
# plt.xlim(3, 4)
plt.xlabel('Vertex energy [MeV/u]')
plt.ylabel('(value – fit) / fit')

In [ ]:
def ruth_transform(x):
#     return (x**-2).reshape(-1, 1)
    return np.column_stack((x**2, x, x**-1, x**-2, x**-3, x**-4, x**-5))

def make_rmat_norms(data):
    return data.sum(('run_num', 'vert_en')) / data.sum()

def process_rmat(rmat, rmat_norms):
    rmat_model = LinearRegression()
    X_ = ruth_transform(rmat.cm_vert_en.values)
    Y_ = (rmat / rmat.max('vert_en') * rmat_norms).sum('cm_angle')

    rmat_model.fit(X_, Y_)

    rmat_baselines = rmat_model.predict(X_)

    r = (Y_ - rmat_baselines) / rmat_baselines
    
    return r, rmat_baselines, Y_

def process_rmat_with_flatcurve(rmat, rmat_norms, baseline):
    Y_ = (rmat / rmat.max('vert_en') * rmat_norms).sum('cm_angle')
    rmat_baselines = (baseline / baseline.max('vert_en') * rmat_norms).sum('cm_angle')
    r = (Y_ - rmat_baselines) / rmat_baselines
    return r, rmat_baselines, Y_

In [ ]:
rmat_norms = make_rmat_norms(binned)
# flatr, rmat_baselines, totr = process_rmat(rmat, rmat_norms)
flatr, rmat_baselines, totr = process_rmat_with_flatcurve(rmat, rmat_norms, rmat_bg)
# flatr_phase, rmat_baselines_phase, totr_phase = process_rmat(rmat_phase, rmat_norms)
flatr_phase, rmat_baselines_phase, totr_phase = process_rmat_with_flatcurve(rmat_phase, rmat_norms, rmat_bg)

In [ ]:
with sns.plotting_context('paper'):
    plt.figure(figsize=(4, 4*2/3))
    plt.plot(rmat_phase.vert_en.values, totr_phase, label='Full calculation')
    plt.plot(rmat_phase.vert_en.values, rmat_baselines, '--', label='Baseline')
    plt.semilogy()
    
    plt.legend(frameon=False)

    plt.xlim(1.9, 4.1)
    plt.ylim(0.9e-2, 1.1e-1)

    plt.xlabel('Proton vertex energy (lab) [MeV]')
    plt.ylabel('Weighted total excitation\nfunction [unitless]')
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/rmat_bgfit.pdf', bbox_inches='tight')

In [ ]:
rmat_norms.plot()

In [ ]:
srimdata = np.array([
    [3.86, 3.44],
    [3.54, 6.36],
    [3.21, 6.15],
    [2.86, 6.87],
    [2.49, 8.56],
    [2.10, 9.33],
    [1.67, 11.24],
])
srimens = srimdata[:, 0]
srimsigmas = srimdata[:, 1] / 1000
plt.plot(srimens, srimsigmas, 'o')
srim_X = sm.add_constant(srimens)
srim_model = sm.OLS(srimsigmas, srim_X)
srim_fit = srim_model.fit()
plt.plot(srimens, srim_fit.predict(srim_X))
print(srim_fit.params)

In [ ]:
sigma_func = lambda en: np.asscalar(np.hypot(0.0197, srim_fit.predict([1, en])))

In [ ]:
xs = np.linspace(2.0, 4.0, 100)
plt.plot(xs, [sigma_func(x) for x in xs])

In [ ]:
def convolve(signal, window):
    result = xr.zeros_like(signal)
    if not callable(window):
        window_func = lambda m, z: window[m]
    else:
        window_func = window
        
    shift = len(window) // 2
    for n in range(len(signal)):
        z = signal.vert_en[n]
        result[n - shift] = np.sum([signal[n-m] * window_func(m, z) for m in range(len(window))])
    
    return result


class VariableWidthWindow:
    def __init__(self, num_pts, binsize, sigma_func):
        self.num_pts = num_pts
        self.sigma_func = sigma_func
        self.binsize = binsize
        
    def __call__(self, m, z):
        sigma = self.sigma_func(z)
        w = scipy.signal.windows.gaussian(self.num_pts, sigma / self.binsize)
        w /= w.sum()
        return w[m]
    
    def __len__(self):
        return self.num_pts

    
def gaus_conv(x, sigma, sigma_func):
    binsize = np.diff(x.vert_en.values)[0]
    
    result = xr.zeros_like(x)

#     w = scipy.signal.windows.gaussian(21, sigma / binsize, sym=True)
#     w /= w.sum()
    w = VariableWidthWindow(21, binsize, sigma_func)
#     result[:] = scipy.signal.convolve(x, w, mode='same')
    result = convolve(x, w)
    
    return result

In [ ]:
# rconv = gaus_conv(flatr, 0.0197)
rconv_phase = gaus_conv(flatr_phase, 0.0197, sigma_func)

In [ ]:
with sns.plotting_context('talk'):
#     plt.figure(figsize=(4, 4*2/3))
#     plt.plot(flatr.vert_en, flatr.values, ':', label='Full resolution')
    plt.plot(flatr_phase.vert_en, flatr_phase.values, ':', label='Full resolution')
#     plt.plot(rconv.vert_en, rconv, '-', label='Convolved with Gaussian')
    plt.plot(rconv_phase.vert_en, rconv_phase, '-', label='Convolved with Gaussian')
    plt.axhline(0, color='k', zorder=0, linewidth=1)

    plt.xlim(1.9, 4.1)
    plt.ylim(-0.110)

    plt.xlabel('Proton vertex energy (lab) [MeV]')
    plt.ylabel('(value – baseline) / baseline')

    plt.legend(loc=3, frameon=False)
    
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/rmat_conv.pdf', bbox_inches='tight')

In [ ]:
# pal = sns.cubehelix_palette(n_colors=len(rmat_ratios.cm_angle), light=0.7)
# with sns.color_palette(pal, len(rmat_ratios.cm_angle)):
#     for i, row in enumerate(rmat_ratios.values.T):
#         plt.plot(rmat_ratios.cm_vert_en, row - i / 5)
#     plt.xlim(2, 4)

In [ ]:
with sns.plotting_context('talk'):
#     plt.figure(figsize=(6.25, 6.25*2/3))
    plt.errorbar(d.vert_en, d.values, derr, elinewidth=0.5, fmt='.', zorder=100, label='Experiment')
#     plt.plot(rconv.vert_en, rconv.values, 'C1--', label='DSigmaIV (0° mixing)')
    plt.plot(rconv_phase.vert_en, rconv_phase.values, '-', label='DSigmaIV')
    plt.axhline(0, color='k', zorder=0, linewidth=1)

    plt.xlim(1.9, 4.1)
    plt.ylim(-0.12, 0.10)

    plt.xlabel('${}^{46}\mathrm{Ar}$ vertex energy (lab) [MeV/u]')
    plt.ylabel('(value – baseline) / baseline')
    
    plt.text(2.76, -0.04, r'$\frac{3}{2}^-$')
    plt.text(3.12, -0.06, r'$\frac{1}{2}^+$')
    plt.text(3.76, -0.07, r'$\frac{1}{2}^-$')
    
#     plt.text(0.95, 0.95, 'PRELIMINARY', transform=plt.gca().transAxes, ha='right',
#              fontdict={'size': 20, 'color': 'C7'})

    plt.legend(loc='lower center', ncol=3)
#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/bgfit_comp.pdf', bbox_inches='tight')
#     plt.savefig('/Users/josh/Desktop/result.pdf', bbox_inches='tight')

In [ ]:
def chi2stat(data, err, rmat):
    spl = InterpolatedUnivariateSpline(rmat.vert_en, rmat.values)
    return np.asscalar(np.sum((data - spl(data.vert_en))**2 / err**2))

def rss_stat(data, rmat):
    spl = InterpolatedUnivariateSpline(rmat.vert_en, rmat.values)
    return np.asscalar(np.sum((data - spl(data.vert_en))**2))

def tss_stat(data):
    return np.asscalar(np.sum(data**2))

def chi2zero(data, err):
    return np.asscalar(np.sum(data**2 / err**2))

def find_fvalue(rss1, rss2, df1, df2):
    num = (rss1 - rss2) / (df1 - df2)
    denom = rss2 / df2
    return num / denom

def make_stat_cut(data, min_, max_):
    return data[(data.vert_en > min_) & (data.vert_en < max_)]

def res_integral(res_data):
    pos_data = res_data[res_data > 0]
    neg_data = res_data[res_data < 0]
    dE = np.diff(res_data.vert_en.values)[0]
    
    return (np.asscalar(x.sum('vert_en') * dE) for x in (pos_data, neg_data))

def outlier_mask(data, num_std):
    return (data - np.mean(data)) < num_std * np.std(data)

def analyze_resonance(stat_min_en, stat_max_en, num_std=None):
    stat_d = make_stat_cut(d, stat_min_en, stat_max_en)
    stat_derr = make_stat_cut(derr, stat_min_en, stat_max_en)
    stat_r = make_stat_cut(rconv_phase, stat_min_en, stat_max_en)
    
    if num_std is not None:
        mask = outlier_mask(stat_d, num_std)
        stat_d = stat_d[mask]
        stat_derr = stat_derr[mask]
    
    plt.plot(stat_r.vert_en, stat_r.values, zorder=100)
    plt.errorbar(stat_d.vert_en, stat_d.values, stat_derr.values, fmt='.', elinewidth=0.5)
    plt.axhline(0, color='k', linewidth=1)

    fit_chi_dof = len(stat_d) - 6
    fit_chi = chi2stat(stat_d, stat_derr, stat_r) / fit_chi_dof 
    print(f'Fit χ^2 = {fit_chi:0.4g}')
    fit_rss = rss_stat(stat_d, stat_r)
    print(f'Fit RSS = {fit_rss:0.4g}')

    null_chi_dof = len(stat_d)
    null_chi = chi2zero(stat_d, stat_derr) / null_chi_dof
    print(f'Null χ^2 = {null_chi:0.4g}')
    null_rss = tss_stat(stat_d)
    print(f'Null RSS = {null_rss:0.4g}')

    # fvalue = null_chi / fit_chi; print(fvalue)
    fvalue = find_fvalue(null_rss, fit_rss, null_chi_dof, fit_chi_dof)
    print(f'F = {fvalue:0.4g}')

    pvalue = fstat.sf(fvalue, null_chi_dof - fit_chi_dof, fit_chi_dof)
    print(f'p = {pvalue:0.4g}')
    
    pos_resint, neg_resint = res_integral(stat_r)
    print(f'resint = {pos_resint:0.4g}(+) / {neg_resint:0.4g}(-)')

In [ ]:
analyze_resonance(2.55, 2.86)

In [ ]:
analyze_resonance(2.86, 3.25)

In [ ]:
analyze_resonance(3.15, 3.55)

In [ ]:
analyze_resonance(3.5, 4)

In [ ]:
analyze_resonance(2.5, 4)

In [ ]:
from scipy.optimize import curve_fit
from functools import partial

class RescaledResonance:
    def __init__(self, rmat_data):
        self.spline = InterpolatedUnivariateSpline(rmat_data.vert_en, rmat_data.values, ext='zeros')
        
    def __call__(self, x, scale=1, width=1, offset=0):
#         rescaled_x = width * (x - ctr) + ctr
#         rescaled_x = width * (x - offset)
        rescaled_x = (x - offset) / (scale * width)
        return self.spline(rescaled_x) * scale

In [ ]:
lb, ub = 3.5, 4.0
rdata = make_stat_cut(rconv_phase, lb, ub)
ddata = make_stat_cut(d, lb, ub)
derrdata = make_stat_cut(derr, lb, ub)
resc = RescaledResonance(rconv_phase)

In [ ]:
# fitfunc = partial(resc, ctr=3.68)
fitfunc = resc

fparams, fcov = curve_fit(fitfunc, ddata.vert_en, ddata.values, p0=[1, 1, 0])
#                           bounds=([-10, -10, -10], [10, 10, 10]))
print(fparams)
# print('offset = {:04g}'.format(fparams[2] * fparams[1]))

In [ ]:
plt.errorbar(ddata.vert_en, ddata.values, derrdata, fmt='.', elinewidth=0.5)

xs = np.linspace(ddata.vert_en.min(), ddata.vert_en.max(), 100)
plt.plot(xs, fitfunc(xs, *fparams))
plt.plot(rdata.vert_en, rdata.values, '--')

# Examining dependence on $\Gamma_\downarrow$

In [ ]:
width_data = pd.read_excel('/Users/josh/Documents/Data/Results/e15503b/resonance_integrals.xlsx', index_col='gdown')
width_data *= 1000

In [ ]:
width_data.plot(style='.-')
plt.xlim(-2, 42)

In [ ]:
width_mdata = pd.melt(width_data.reset_index(), id_vars=['gdown'], var_name='resonance', value_name='resint')

In [ ]:
def plot_resint_fit(xdata, ydata, color=None):
#     def feat_trans(data):
#         data = np.asarray(data, dtype='float')
#         return np.column_stack((data, data**2, data**3, data**4))
    
    model = LinearRegression(fit_intercept=True)
    
    model.fit(xdata.values.reshape(-1, 1), ydata)
    plt.scatter(xdata, ydata)
    xs = np.linspace(xdata.min(), xdata.max(), 100)
    plt.plot(xs, model.predict(xs.reshape(-1, 1)), 'C1')
    params_text = AnchoredText('$y = {m:0.3g} x {b:+0.3g}$'.format(m=model.coef_[0], b=model.intercept_), 1)
    plt.gca().add_artist(params_text)

fg = sns.FacetGrid(width_mdata, col='resonance', col_wrap=2, sharey=True)
fg.map(plot_resint_fit, 'gdown', 'resint')
fg.axes[0].set_ylim(-8, 6)
fg.set_axis_labels(r'$\Gamma_\downarrow$ [keV]', r'Resonance integral ($\times 1000$)')
fg.set_titles(r'{col_name}')

In [ ]:
(width_data.loc[50] - width_data.loc[40])

In [ ]:
width_data

In [ ]:
def dGamma_p(gd, I, dIdgd):
    num = (dIdgd * gd + I) * (2*pi - I) + dIdgd * I * gd
    denom = (2*pi - I)**2
    return num / denom

In [ ]:
dGamma_p(+10, width_data.loc[10, "1/2+ (neg)"] / 1000, 0.015 / 1000)

# Value comparisons

In [ ]:
def comb_err(e1, e2):
    return np.hypot(e1, e2).reshape(-1, 1)

In [ ]:
with sns.plotting_context('paper'):
    fig, ax = plt.subplots(2, 2, sharey=True, figsize=(4.5, 4.5))

    pw   = dict(fmt='o', color='C0', label='Present work')
    gaud = dict(fmt='s', color='C1', label='Gaudefroy et al.')
    bhat = dict(fmt='D', color='C2', label='Bhattacharyya et al.')
    sdpfmu = dict(fmt='^', markeredgecolor='C3', color='w', markeredgewidth=1, label='SDPF-MU [Gade et al.]')
    sdpfu  = dict(fmt='v', markeredgecolor='C4', color='w', markeredgewidth=1, label='SDPF-U [Gade et al.]')
    sdpf   = dict(fmt='>', markeredgecolor='C5', color='w', markeredgewidth=1, label='SDPF [Gaudefroy et al.]')

    source_map = {
        'pw': (pw, 0),
        'gaud': (gaud, 1),
        'bhat': (bhat, 2),
        'sdpf': (sdpf, 5),
        'sdpfu': (sdpfu, 4),
        'sdpfmu': (sdpfmu, 3),
    }

    def plot_value(value, err, source, axes=None):
        if axes is None:
            axes = plt.gca()

        source_opts, source_index = source_map[source]
        axes.errorbar(value, source_index, xerr=err, **source_opts)

    def add_label(text, axes):
        label = AnchoredText(text, loc=2, frameon=False)
        axes.add_artist(label)

    # 3/2- energy
    plot_value(0, comb_err([28, 28], [20, 25]), 'pw', ax[0, 0])  # pw
    plot_value(0, 0, 'gaud', ax[0, 0])
    plot_value(0, 0, 'bhat', ax[0, 0])
    plot_value(0, 0, 'sdpfmu', ax[0, 0])
    plot_value(0, 0, 'sdpfu', ax[0, 0])
    plot_value(0, 0, 'sdpf', ax[0, 0])
    add_label(r'$3/2^-$', ax[0, 0])

    # 1/2- energy
    plot_value(970, comb_err([28, 28], [50, 10]), 'pw', ax[1, 0])  # pw
    plot_value(1200, 6, 'bhat', ax[1, 0])  # Bhatta.
    plot_value(1130, 75, 'gaud', ax[1, 0])  # Gaud.
    plot_value(1139, 0, 'sdpfu', ax[1, 0])
    plot_value(931, 0, 'sdpfmu', ax[1, 0])
    plot_value(1251, 0, 'sdpf', ax[1, 0])
    add_label(r'$1/2^-$', ax[1, 0])

    # 3/2- SF
    plot_value(0.27, comb_err([0.03, 0.03], [0.13, 0.21]), 'pw', ax[0, 1])  # pw
    plot_value(0.711, 0, 'sdpfu', ax[0, 1])
    plot_value(0.634, 0, 'sdpfmu', ax[0, 1])
    plot_value(0.61, 0.05, 'gaud', ax[0, 1])  # Gaudefroy
    plot_value(0.64, 0, 'sdpf', ax[0, 1])
    ax[0, 1].set_xlim(-0.1, 1.1)
    add_label(r'$3/2^-$', ax[0, 1])

    # 1/2- SF
    plot_value(0.42, comb_err(0.05, 0.09), 'pw', ax[1, 1])  # pw
    plot_value(0.81, 0.06, 'gaud', ax[1, 1])  # Gaud.
    plot_value(0.834, 0, 'sdpfu', ax[1, 1])
    plot_value(0.931, 0, 'sdpfmu', ax[1, 1])
    plot_value(0.81, 0, 'sdpf', ax[1, 1])
    ax[1, 1].set_xlim(-0.1, 1.1)
    add_label(r'$1/2^-$', ax[1, 1])

    ax[1, 0].set_yticks([])
    # sns.despine(left=True)

    # Labels
    # ax[0, 0].set_ylabel(r'$3/2^-$', rotation=0, ha='right', va='center')
    # ax[1, 0].set_ylabel(r'$1/2^-$', rotation=0, ha='right', va='center')
    ax[1, 0].set_xlabel(r'$E_x$ [keV]')
    ax[1, 1].set_xlabel(r'$S$')
    ax[0, 0].set_xlabel(r'$E_x$ [keV]')
    ax[0, 1].set_xlabel(r'$S$')
    
    ax[0, 0].set_ylim(-0.5, 5.5)

    leg_handles, leg_labels = ax[0, 0].get_legend_handles_labels()
    legend = fig.legend(leg_handles, leg_labels, loc='upper center', bbox_to_anchor=(0.5, 0.14), ncol=2)
    plt.tight_layout(rect=(0, 0.10, 1, 1), h_pad=0.2)

#     plt.savefig('/Users/josh/Documents/Papers/phd-thesis/Figures/exp/lit_comp.pdf', bbox_inches='tight')

In [ ]:
leg_labels

# Fit plotting

In [ ]:
from pytpc.fitting import MCFitter
import yaml

with open('../fitters/config_e15503b.yml', 'r') as f:
    config = yaml.load(f)
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    patch = yaml.load(f)
config.update(patch)

fitter = MCFitter(config)

In [ ]:
class FitPlotter(object):
    def __init__(self, data_path, mcfitter):
        self.data_path = data_path
        self.fitter = mcfitter
        
    def plot_fit(self, fitres, run_num, evt_id):
        fp = os.path.join(self.data_path, 'clean_run_{:04d}.h5'.format(run_num))
        with h5py.File(fp, 'r') as hf:
            dataset = hf['/clean/{}'.format(int(evt_id))]
            raw_xyz = dataset[:]
            cx, cy = dataset.attrs['center'][:2]
            
        raw_xyz = pytpc.cleaning.apply_clean_cut(raw_xyz)
        xyz, (cu, cv) = self.fitter.preprocess(raw_xyz, center=[cx, cy], rotate_pads=False, last_tb=505)
#         xyz = pytpc.cleaning.nn_remove_noise(xyz)
        print('Num points:', len(xyz))
        
        tr = self.fitter.tracker.track_particle(fitres.x0, fitres.y0, fitres.z0, fitres.enu0, 
                                                fitres.azi0, fitres.pol0)
        
        arclen = np.sum(np.sqrt(np.sum(np.diff(tr[:, :3], axis=0)**2, -1)))
        print('Arclen:', arclen)
        print('Energy from arclen: ', self.fitter.gas.inverse_range(arclen, 1, 1))
        
        orig_guess = self.fitter.guess_parameters(fitres)
        orig_tr = self.fitter.tracker.track_particle(*orig_guess)
        
        xyz['cx'] = xyz.u - cu
        xyz['cy'] = xyz.v - cv
        xyz['cr'] = np.hypot(xyz.cx, xyz.cy)
        xyz['cth'] = np.unwrap(constrain_angle(np.arctan2(-cv, -cu) - np.arctan2(xyz.cy, xyz.cx)), discont=1.8 * pi)
        if np.abs(xyz['cth'].min() - 2 * pi) < 5 * degrees:
            xyz['cth'] -= 2 * pi
        
        fig, ax = plt.subplots(3, 2, figsize=(12, 14))
        fig.subplots_adjust(hspace=0.4, wspace=0.4)
        ax[0, 0].plot(xyz.w / 1000, xyz.u / 1000, '.')
        ax[0, 0].plot(tr[:, 2], tr[:, 0])
        ax[0, 0].plot(orig_tr[:, 2], orig_tr[:, 0], '--')
        ax[0, 0].set_xlabel('w [m]')
        ax[0, 0].set_ylabel('u [m]')
        
        ax[0, 1].plot(xyz.w / 1000, xyz.v / 1000, '.')
        ax[0, 1].plot(tr[:, 2], tr[:, 1])
        ax[0, 1].plot(orig_tr[:, 2], orig_tr[:, 1], '--')
        ax[0, 1].set_xlabel('w [m]')
        ax[0, 1].set_ylabel('v [m]')
        
        ax[1, 0].plot(xyz.w/1000, np.hypot(xyz.u, xyz.v) / 1000, '.')
        ax[1, 0].plot(tr[:, 2], np.hypot(tr[:, 0], tr[:, 1]))
        ax[1, 0].plot(orig_tr[:, 2], np.hypot(orig_tr[:, 0], orig_tr[:, 1]), '--')
        ax[1, 0].set_xlabel('w [m]')
        ax[1, 0].set_ylabel('$\sqrt{u^2 + v^2}$ [m]')
        
        ax[1, 1].plot(xyz.u / 1000, xyz.v / 1000, '.')
        ax[1, 1].plot(tr[:, 0], tr[:, 1])
        ths = np.linspace(0, 2*pi, 100)
        ax[1, 1].plot(fitres.rad_curv/1000 * np.cos(ths) + fitres.curv_ctr_x/1000, 
                      fitres.rad_curv/1000 * np.sin(ths) + fitres.curv_ctr_y/1000, 'r--')
        ax[1, 1].plot(cu/1000, cv/1000, 'k+', mew=1.5)
        ax[1, 1].plot(fitres.curv_ctr_x/1000, fitres.curv_ctr_y/1000, 'ro')
        ax[1, 1].set_aspect(1)
        ax[1, 1].set_xlabel('u [m]')
        ax[1, 1].set_ylabel('v [m]')
        
        slope = np.tan(fitres.lin_scat_ang - pi / 2)
        ydata = xyz.cr * xyz.cth
        ax[2, 0].plot(ydata, xyz.w, '.')
#         ax[2, 0].plot(ydata, odrline([fitres.lin_beam_int, slope], ydata))
        
        ax[2, 1].plot(xyz.w / 1000, xyz.a, '.')

In [ ]:
fitpl = FitPlotter('/Users/josh/Documents/Data/Merged/e15503b/clean', fitter)

In [ ]:
i = 0

In [ ]:
i -= 1

In [ ]:
e = 4.0
de = 0.05
idx = res[goodfitcut & physicalcut & prodcut
          #& (res.enu0 < 1.2)
#           & (res.run_num == 115)
#           & (res.evt_id == 563)
#           & res.run_num.isin([76, 85, 99, 193, 263])
          & (res.vert_en > 1) & (res.vert_en < 2)
#           & (res.enu0 < 2)
          & (res.cm_angle > 80*degrees)# & (res.cm_angle < 45*degrees)
#           & (res.run_num > 217)
#           & (res.posChi2 < posChi2Max)
#           & (res.enChi2 < enChi2Max)
#         (res.enChi2 > 15) & (res.enChi2 < 30) 
].sample(1).index[0]

fr = res.loc[idx]

print('Run {} event {}'.format(int(fr.run_num), int(fr.evt_id)))
print('linfit chi2:', fr.lin_chi2)
print('mcmin poschi2:   ', fr.posChi2)
print('mcmin  enchi2:   ', fr.enChi2)
print('kine vert_en:    ', fr.kine_vert_en)
print('z0 vert_en:      ', fr.vert_en)
print('z0 vert_en_corr: ', fr.vert_en)
print('ctr:', fr[['x0', 'y0', 'z0', 'enu0', 'azi0', 'pol0']].values)
print('CM Angle: {:0.2f}   Lab angle: {:0.2f}'.format(fr.cm_angle / degrees, fr.scat_ang / degrees))
fitpl.plot_fit(fr, int(fr.run_num), fr.evt_id)

In [ ]:
X, Y = np.meshgrid(binned.cm_angle, binned.vert_en)
plt.pcolormesh(X, Y, binned.sum('run_num').values, norm=LogNorm())
plt.scatter(fr.cm_angle / degrees, fr.vert_en, c='C1', marker='x', s=200)

In [ ]:
frd = fr.to_dict()
del frd['brho']
frd

In [ ]:
rn = int(fr.run_num)
with pytpc.HDFDataFile(
        f'/Volumes/attpc/data/e15503b/full-trace/run_{rn:04d}.h5', open_mode='r', 
        canonical_evtid_path=f'/Volumes/attpc/data/e15503b/canon_ids/canon_ids_run_{rn:04d}.h5') as hf:
    evt = hf[int(fr.evt_id)]

In [ ]:
from pytpc.evtdata import fix_baselines

In [ ]:
traces = fix_baselines(evt.traces['data'])
for tr in traces:
    plt.plot(tr)

In [ ]:
with h5py.File('/Users/josh/Documents/Data/Merged/e15503b/clean/clean_run_{:04d}.h5'.format(int(fr.run_num)), 'r') as f:
    raw_xyz = f['/clean/{}'.format(int(fr.evt_id))][:]
raw_xyz = apply_clean_cut(raw_xyz)
xyz = fitter.preprocess(raw_xyz)

In [ ]:
res_track = fitter.tracker.track_particle(fr.x0, fr.y0, fr.z0, fr.enu0, fr.azi0, fr.pol0)

In [ ]:
exp_hits = np.zeros(10240)
for a, p in xyz[['a', 'pad']].values:
    exp_hits[int(p)] += a

In [ ]:
fig, sm = pad_plot(exp_hits, scale='linear', cmap='viridis')
plt.colorbar(sm)

In [ ]:
exp_mesh = np.zeros(512)
for z, a in raw_xyz[:, 2:4]:
    exp_mesh += transfer(a, 280./80, z)
    
res_mesh = fitter.evtgen.make_mesh_signal(res_track[:, :3].copy(), res_track[:, 4].copy())

In [ ]:
plt.plot(res_track[:, 2]*1000, res_track[:, 1]*1000, 'k', zorder=100)
plt.plot(xyz.w, xyz.v, '.')
plt.xlim(0, 1000)
plt.hlines(0, *plt.xlim(), color='g')
plt.ylim(-275, 275)
plt.gca().set_aspect(1)
plt.xlabel('z [mm]')
plt.ylabel('y [mm]')
plt.title('Run {:.0f} event {:.0f}'.format(fr.run_num, fr.evt_id))
# plt.savefig('/Users/josh/Desktop/plots_for_daniel/yz.pdf', bbox_inches='tight')

In [ ]:
plt.plot(res_track[:, 2]*1000, res_track[:, 0]*1000, 'k', zorder=100)
plt.plot(xyz.w, xyz.u, '.')
plt.xlim(0, 1000)
plt.hlines(0, *plt.xlim(), color='g')
plt.ylim(-275, 275)
plt.gca().set_aspect(1)
plt.xlabel('z [mm]')
plt.ylabel('x [mm]')
plt.title('Run {:.0f} event {:.0f}'.format(fr.run_num, fr.evt_id))
# plt.savefig('/Users/josh/Desktop/plots_for_daniel/xz.pdf', bbox_inches='tight')

In [ ]:
plt.plot(res_track[:, 0]*1000, res_track[:, 1]*1000, 'k', zorder=100)
plt.plot(xyz.u, xyz.v, '.')
plt.xlim(-275, 275)
plt.ylim(-275, 275)
plt.gca().set_aspect(1)
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.title('Run {:.0f} event {:.0f}'.format(fr.run_num, fr.evt_id))
# plt.savefig('/Users/josh/Desktop/plots_for_daniel/xy.pdf', bbox_inches='tight')

In [ ]:
with sns.axes_style('ticks'), sns.plotting_context('paper'):
    plt.figure(figsize=(3.5, 3.5))
    plt.plot(res_track[:, 0]*1000, res_track[:, 1]*1000, 'k', zorder=100)
    plt.plot(xyz.u, xyz.v, '.')
    plt.gca().set_aspect(1)
    
    sns.despine()
    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')
    plt.tight_layout()
    
#     plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/evt_{:04.0f}_{:0.0f}_xy.pdf'.format(fr.run_num, fr.evt_id))

In [ ]:
with sns.axes_style('ticks'), sns.plotting_context('paper'): 
    plt.figure(figsize=(3.5, 2.4))
    plt.plot(res_track[:, 2]*1000, res_track[:, 1]*1000, 'k', zorder=100)
    plt.plot(xyz.w, xyz.v, '.')
    
    sns.despine()
    plt.xlabel('z [mm]')
    plt.ylabel('y [mm]')
    plt.tight_layout()
    
    plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/evt_{:04.0f}_{:0.0f}_zy.pdf'.format(fr.run_num, fr.evt_id))

In [ ]:
%matplotlib inline

In [ ]:
with sns.axes_style('white'):
    plt.subplot(111, projection='3d')
    plt.plot(res_track[:, 0]*1000, res_track[:, 1]*1000, res_track[:, 2]*1000, 'k')
    plt.plot((0, 0), (0, 0), (0, 1000), 'g')
    plt.gca().scatter(xyz.u, xyz.v, xyz.w, linewidths=0)
    plt.xlim(-275, 275)
    plt.ylim(-275, 275)
    plt.gca().set_zlim(0, 1000)
    plt.gca().set_aspect(1)
#     plt.savefig('/Users/josh/Desktop/plots_for_daniel/3d.pdf', bbox_inches='tight')

In [ ]:
fr

In [ ]:
np.rad2deg(fr.pol0) - 180